In [95]:
import os
import argparse
import sys
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import torch.optim as optim
import torch.nn as nn
from testloader import iris_dataload

In [96]:
# parser = argparse.ArgumentParser()
# parser.add_argument('--num_classes', type=int, default=100, help='the number of classes')
# parser.add_argument('--epochs', type=int, default=20, help='the number of training epoch')
# parser.add_argument('--batch_size', type=int, default=16, help='batch_size for training')
# parser.add_argument('--lr', type=float, default=0.005, help='star learning rate')   
# parser.add_argument('--data_path', type=str, default="神经网络实现鸢尾花分类\Iris_data.txt") 
# parser.add_argument('--device', default='cuda', help='device id (i.e. 0 or 0,1 or cpu)')
# opt = parser.parse_args()  


In [97]:
class Neuralnetwork(nn.Module):
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):
        super(Neuralnetwork,self).__init__()
        self.layer1 = nn.Linear(in_dim, n_hidden_1)
        self.layer2 = nn.Linear(n_hidden_1, n_hidden_2)
        self.layer3 = nn.Linear(n_hidden_2, out_dim)
    
    def forward(self,x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return(x)
    


In [98]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [99]:
custom_dataset = iris_dataload( "./Iris_data.txt") 
train_size = int(len(custom_dataset)*0.7)
validate_size = int(len(custom_dataset)*0.2)
test_size = len(custom_dataset) - train_size -validate_size
train_dataset,validate_dataset,test_dataset = torch.utils.data.random_split(custom_dataset,[train_size,validate_size,test_size])

train_loader = DataLoader(train_dataset,batch_size=16,shuffle=False)
validate_loader= DataLoader(validate_dataset,batch_size=1,shuffle=False)
test_loader= DataLoader(test_dataset,batch_size=1,shuffle=False)
print("Training set data size:", len(train_loader)*16, ",Validating set data size:", len(validate_loader), ",Testing set data size:", len(test_loader)) 

150images were found in the dataset
Training set data size: 112 ,Validating set data size: 30 ,Testing set data size: 15


In [100]:
#定义推理过程，返回准确率，用于验证阶段和测试阶段
def infer(model,dataset,device):
    model.eval()
    acc_num =0.0
    with torch.no_grad():
        for data in dataset:
            datas,labels = data
            outputs = model(datas.to(device))
            predict_y = torch.max(outputs,dim=1)[1]
            acc_num += torch.eq(predict_y,labels.to(device)).sum().item()
    accuratcy = acc_num/len(dataset)
    return accuratcy


In [103]:
#定义训练，验证，测试阶段
def main():

    model = Neuralnetwork(4,12,6,3).to(device)
    loss_fuction = nn.CrossEntropyLoss()
    pg = [p for p in model.parameters() if p.requires_grad]
    optimizer = optim.Adam(pg, lr = 0.001)

    #定义模型权重存储地址
    save_path = os.path.join(os.getcwd(),'test_results/weights')
    if os.path.exists(save_path) is False:
        os.makedirs(save_path)

    epochs = 20
    #开始训练过程
    for epoch in range(epochs):
        ############################################################## train ######################################################
        model.train()
        acc_num = torch.zeros(1).to(device)
        sample_num = 0
        train_bar = tqdm(train_loader,file =sys.stdout,ncols = 100)
        for datas in train_bar:
            data,label = datas
            label = label.squeeze(-1)
            sample_num += data.shape[0]

            optimizer.zero_grad()
            outputs = model(data.to(device))# output_shape: [batch_size, num_classes]
            pred_class = torch.max(outputs,dim=1)[1]# torch.max 返回值是一个tuple，第一个元素是max值，第二个元素是max值的索引。
            acc_num += torch.eq(pred_class,label.to(device)).sum()

            loss = loss_fuction(outputs,label.to(device))

            loss.backward()
            optimizer.step()

            train_acc = acc_num.item()/sample_num
            train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch+1,epochs,loss)
        ############################################################## validate ###################################################### 
        val_acc = infer(model = model ,dataset=validate_loader,device=device)
        print('[epoch %d] train_loss: %.3f  train_acc: %.3f  val_accuracy: %.3f' %  (epoch + 1, loss, train_acc, val_acc))   
        torch.save(model.state_dict(),os.path.join(save_path,"test_NN_pth"))

        train_acc =0.0
        val_acc = 0.0
    print('完成')
    ################################################################# test ############################################################         
    test_acc = infer(model = model,dataset=test_loader,device=device)
    print('test_accuracy:%.3f'%(test_acc))

if __name__=="main":
    main()

In [104]:
main()

train epoch[1/20] loss:1.167: 100%|██████████████████████████████████| 7/7 [00:00<00:00, 595.25it/s]
[epoch 1] train_loss: 1.167  train_acc: 0.314  val_accuracy: 0.367
train epoch[2/20] loss:1.131: 100%|██████████████████████████████████| 7/7 [00:00<00:00, 466.74it/s]
[epoch 2] train_loss: 1.131  train_acc: 0.352  val_accuracy: 0.367
train epoch[3/20] loss:1.098: 100%|██████████████████████████████████| 7/7 [00:00<00:00, 999.97it/s]
[epoch 3] train_loss: 1.098  train_acc: 0.362  val_accuracy: 0.367
train epoch[4/20] loss:1.068: 100%|██████████████████████████████████| 7/7 [00:00<00:00, 874.96it/s]
[epoch 4] train_loss: 1.068  train_acc: 0.362  val_accuracy: 0.367
train epoch[5/20] loss:1.040: 100%|██████████████████████████████████| 7/7 [00:00<00:00, 777.53it/s]
[epoch 5] train_loss: 1.040  train_acc: 0.362  val_accuracy: 0.367
train epoch[6/20] loss:1.015: 100%|██████████████████████████████████| 7/7 [00:00<00:00, 699.93it/s]
[epoch 6] train_loss: 1.015  train_acc: 0.362  val_accuracy